# Setup

In [23]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
t = tf.constant([[1, 2, 3], [4, 5, 6]], dtype=tf.float32)

In [3]:
t.shape

TensorShape([2, 3])

In [4]:
t.dtype

tf.float32

In [5]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [6]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [7]:
t+10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [8]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [9]:
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

# Creating loss function

In [24]:
import tensorflow.keras.backend as K

K.clear_session()
np.random.seed(55)
tf.random.set_seed(55)

In [25]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target.reshape(-1, 1))
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [4]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss  = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [16]:
input_shape = X_train[0].shape

model = Sequential()
model.add(Dense(100, activation='selu', kernel_initializer='lecun_normal', input_shape=input_shape))
model.add(Dense(1, activation='relu'))

In [17]:
model.compile(loss=huber_fn, optimizer='nadam', metrics=['mae', 'mse'])

In [18]:
history = model.fit(X_train_scaled, y_train, validation_data=(X_valid_scaled, y_valid), 
                    epochs=20, callbacks=[keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True)])

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4005 - mae: 0.7376 - mse: 1.1009 - val_loss: 0.1954 - val_mae: 0.4951 - val_mse: 0.4344
Epoch 2/20
363/363 [==============================] - 1s 2ms/step - loss: 0.1960 - mae: 0.4870 - mse: 0.4510 - val_loss: 0.1858 - val_mae: 0.4744 - val_mse: 0.4180
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 0.1913 - mae: 0.4789 - mse: 0.4397 - val_loss: 0.1787 - val_mae: 0.4614 - val_mse: 0.4023
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 0.1869 - mae: 0.4706 - mse: 0.4300 - val_loss: 0.1760 - val_mae: 0.4584 - val_mse: 0.3948
Epoch 5/20
363/363 [==============================] - 1s 2ms/step - loss: 0.1841 - mae: 0.4662 - mse: 0.4236 - val_loss: 0.1752 - val_mae: 0.4560 - val_mse: 0.3942
Epoch 6/20
363/363 [==============================] - 1s 2ms/step - loss: 0.1814 - mae: 0.4616 - mse: 0.4167 - val_loss: 0.1727 - val_mae: 0.4542 - val_mse: 0.3860
Epoch 7/20
363/3

In [19]:
model.evaluate(X_test_scaled, y_test)

162/162 [==============================] - 0s 1ms/step - loss: 0.1653 - mae: 0.4288 - mse: 0.3887


[0.1652783304452896, 0.42883214354515076, 0.3886526823043823]

In [20]:
model.save("my_model_with_a_custom_loss.h5")

In [21]:
model = keras.models.load_model("my_model_with_a_custom_loss.h5", 
                               custom_objects={"huber_fn": huber_fn})

In [22]:
model.evaluate(X_test_scaled, y_test)

162/162 [==============================] - 0s 1ms/step - loss: 0.1653 - mean_absolute_error: 0.4288 - mean_squared_error: 0.3887


[0.1652783304452896, 0.42883214354515076, 0.3886526823043823]

In [23]:
type(model.losses)

list

In [26]:
len(model.losses)

0

In [27]:
model.trainable_variables

[<tf.Variable 'dense_2/kernel:0' shape=(8, 100) dtype=float32, numpy=
 array([[ 2.36275986e-01, -5.12928724e-01,  1.34335041e-01,
          2.68945664e-01, -1.88501328e-01, -3.73558700e-01,
         -3.85248870e-01,  5.48124790e-01,  2.67049849e-01,
          1.79819874e-02,  3.72605056e-01,  2.85061419e-01,
         -2.30700206e-02,  2.26347283e-01, -4.82142955e-01,
          2.71364562e-02, -1.63720578e-01, -9.13864642e-05,
          1.04770670e-02, -5.12238741e-01,  3.93989503e-01,
          5.36532626e-02, -1.12724543e+00, -4.86179113e-01,
         -5.40796816e-02, -1.12209052e-01,  1.89118043e-01,
          2.49462724e-01, -1.64259732e+00, -2.83835590e-01,
         -3.42238843e-01, -4.04043704e-01, -2.13023841e-01,
          2.11039022e-01,  7.01865911e-01, -6.17860138e-01,
         -9.08844650e-01, -9.37293828e-01, -3.24794501e-01,
          4.43134874e-01, -1.02472179e-01,  1.94905192e-01,
         -1.36909962e-01, -3.33813101e-01,  3.99635047e-01,
          1.59775764e-01, -9.1

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               900       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 31,301
Trainable params: 31,301
Non-trainable params: 0
_________________________________________________________________


# Exercise

### Custom Layer

In [26]:
class CustomNormalization(keras.layers.Layer):
    def __init__(self, 
               eps=0.001, 
               **kwargs):
        super().__init__(**kwargs)
        self.eps = eps
        
    def build(self, input_shape):
        self.alpha = self.add_weight(
        shape=input_shape[-1:], 
        dtype=tf.float32, 
        initializer="ones")
        
        self.beta = self.add_weight(
        shape=input_shape[-1:], 
        dtype=tf.float32, 
        initializer="zeros")
        
        super().build(input_shape)
        
    def call(self, X):
        mean, variance = tf.nn.moments(X, axes=-1, keepdims=True)
        stddev = tf.sqrt(variance + self.eps)
        return self.alpha * (X-mean)/(stddev) + self.beta
    
    def compute_output_shape(self, input_shape):
        return input_shape
    
    def get_config(self):
        super_config = super().get_config()
        return {**super_config, 
               "eps":self.eps}

In [27]:
X = X_train.astype(np.float32)

custom_layer_norm = CustomNormalization()
keras_layer_norm = keras.layers.LayerNormalization()

tf.reduce_mean(keras.losses.mean_absolute_error(
    keras_layer_norm(X), custom_layer_norm(X)))

<tf.Tensor: shape=(), dtype=float32, numpy=5.6264494e-08>